In [1]:
import wikipedia
wikipedia.set_lang('ja')
p = None
# p = wikipedia.page('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)')
# p = wikipedia.page('ウレロ☆シリーズ')
p = wikipedia.page(pageid=2354614) #'勇者ヨシヒコと魔王の城',
# p = wikipedia.page('怪奇恋愛作戦')
# p = wikipedia.page('アルコ&ピースのオールナイトニッポンシリーズ')
# p = wikipedia.page(pageid=129204)
# p = wikipedia.page('水曜どうでしょう')
# p = wikipedia.page('おぎやはぎのメガネびいき')
# p = wikipedia.page('リーガルハイ')
# p = wikipedia.page('リバースエッジ大川端探偵社')
# p = wikipedia.page('SPEC〜警視庁公安部公安第五課 未詳事件特別対策係事件簿〜')
# p = wikipedia.page('勝手にふるえてろ')
# p = wikipedia.page('ストロベリーナイト(テレビドラマ)')
# p = wikipedia.page('アイアンマン (映画)')
# print p.html()
# print p

In [2]:
import re
import numpy as np
import wikipedia
import bs4.element as bs4elem
from bs4 import BeautifulSoup

# todo handle English+漢字, '-.,' in the name
EXCEPTION = [u'A・T・C事務所',u'Scope co.,ltd.', u'4-Legs',  u'オア・グローリー神宮前店', u'インダストリアル・ライト&マジック']

# convert unicode to utf-8 if data is unicode
def utf(data):
    if isinstance(data, unicode):
        data = data.encode('utf-8')
    return data

# return true if data include only Katakana and alphabet
def isKatakanaOrEng(data):
    for d in data:
        if not ( '゠' <= utf(d) <= 'ヿ' or re.search('\w', utf(d)) ):
            return False
    
    return True

class WikiScraper(object):
    def __init__(self):
        self._name = ''
        self._page = ''
        self._bsObj = None
        self._headlines = []
        self._headline_tag = 'h3'
        self._last_key = ''
        self._result = dict()

    # return string of 'key : value \n'
    def __str__(self):
        output = self._name + '\n'
        for k, v in self._result.items():
            output +=  k + ' : ' 
            for staff in set(v):
                output += staff + ',       '
            output += u'\n'
        return output.encode('utf-8')
    
    # load html data from wikipedia
    # name: title of wikipage
    # lang: wikipedia language
    # pageid : id of wikipage.
    def load_wiki(self, name, lang='ja', pageid=np.nan):
        wikipedia.set_lang(lang)
        try:
            if np.isnan(pageid):
                self._page = wikipedia.page(name)
            else:
                self._page = wikipedia.page(pageid=int(pageid))
            self.load_html(name, self._page.html())
        except:
            print('Can not open wikipage of '+name+' with lang '+lang)    
    
    # load html
    # name: title of wikipage
    # html: html string
    def load_html(self, name, html):
        self._bsObj = BeautifulSoup(html, "html.parser", from_encoding='utf-8')   
        self._name = name
    
    # find headline
    # name name of headline, e.g. スタッフ
    def find_headline(self, name):
        headers = self._bsObj.find_all(class_='mw-headline')
        self._headlines = []
        for h in headers:
            if re.search(name, h.text):
#                 print 'findheadline------------', h.parentname
                self._headline_tag = h.parent.name
                self._headlines.append(h)

    # cleauup text
    # wrapper function
    # remove brackets, blank, and \n
    def cleanup_text(func):
        def wrapper(self, key, data):
            value = []
            data = data.encode('utf-8')
            data = re.sub('(［|\[).+?(］|\])','', data) #remove brackets
            data = re.sub('(（|\(|\（).+?(）|\）|\))','', data) #remove brackets
            data = data.strip()  #remove \n, and blank from head and tail
            
            #exception
            for e in EXCEPTION:
                if e.encode('utf-8') in data:
                    value.append(e)
                    data = data.replace(e.encode('utf-8'),'')
            
            value = func(key, data, value)
            key = key.strip()
            if key in self._result:
                self._result[key].extend(value)
            else:
                 self._result[key] = value

        return wrapper

    # cleanup value
    # separate line with , / -> and etc. 
    @cleanup_text
    def cleanup_value_list(key, data, value):
        data = re.split('-', data)[0]
        
        data = re.split(u'[,、/／→\n]', data.decode('utf-8')) #separate multiple staffs    
        for d in data:
            d = d.strip()
            if not isKatakanaOrEng(d): #separate with '・' for ウレロシリーズ
                value.extend(re.split(u'[・]', d)) #separate multiple staffs    
#             value.extend(re.split(u'[,、/／→\n]', data.decode('utf-8'))) #separate multiple staffs    
            else:
                value.append(d) # do not separate with '・' for foregin people
#             value.extend(re.split(u'[,、/・／→\n]', data.decode('utf-8'))) #separate multiple staffs    

        for i, v in enumerate(value): #remove ※
            value[i]  = re.split(u'※|●', v)[0].strip()
        
        return value

    # cleanup value
    # separate line with , / -> and etc. 
    @cleanup_text
    def cleanup_value_infobox(key, data, value):
        data = re.split(u'[※|●]', data)[0]
        data = re.split(u'[,、/→\n]', data.decode('utf-8')) #separate multiple staffs    
        for d in data:
            d = d.strip()
            if not isKatakanaOrEng(d): #separate with '・' for ウレロシリーズ
                value.extend(re.split(u'[・]', d)) #separate multiple staffs    
            else:
                value.append(d) # do not separate with '・' for foregin people

        return value
    
    # get list data
    def find_list(self, elt, parent=''):
        # recursive process if there are dl or ul
        for e in elt.find_all(['dl', 'ul']):
#             print e.parent.text
            if e.parent is not None:
                temp =  e.parent.text.split('\n')[0]
                if temp:
                    # researve text as parents
                    parent += e.parent.text.split('\n')[0] + ' '
                    self._last_key = parent
#             for s in e.parent.strings:
#                 parent += s.strip() + ' '
#                 self._last_key = parent
# #                 print parent
#                 break            
            self.find_list(e, parent)
            parent = ''
            if e.parent is not None:
                e.parent.decompose()
        
        # get data from each dd, li or dt
        for e in elt.find_all(['dd', 'li', 'dt']):
            data = e.text
            
            if e.name == 'dd':
                # add data if there is data already ref:朝井リョウ・加藤千恵のオールナイトニッポン staff
                if self._result.get(self._last_key) is not None:
                    data = self._result[self._last_key][-1]+ data
                key = self._last_key
                self._last_value = ''

            elif e.name == 'dt':
                # researve data as a parent
                parent = data
                self._last_key = parent
                continue
                
            elif not re.search(u'[:：-＝\-\=]',data): 
                # if data do not have :, - and etc, i.e. the separator of role and people, save data as a parent
                if parent == '':
                    parent = self._last_key
                key = parent.strip()
                self._last_value = ''

            else:
                #temp for 水曜どうでしょう　& アルピーANN
                if parent != '':
                    # if parents is not empty, set parent as key
                    key = parent
                    if re.search(u'：',data):
                        #if ':' is in data, split and data[0] is key and others will be data
                        datas = re.split(u'：', data)
                        key += datas[0]
                        
                        # connect data except for data[0] in case data has multiple ':'
                        data_temp = ''
                        for d in datas[1:]:
                            data_temp += d
                        data = data_temp
                    
                    elif re.search(u'-',data):
                        # separate data with '-'  and set first one as data, e.g. アルコ&ピースのANN
                        data = re.split(u'-', data)
                        key = parent
                        data = data[0]
                        
                else:
                    if re.search(u'[:：-＝\-\=]',data): 
                        # if parent is empty, separate data and set [0] as key and others as data
                        datas = re.split(u'[:：-＝\-\=]', data)
                        key = datas[0].strip() 
                        
                        data_temp = ''
                        for d in datas[1:]:
                            data_temp += d
                        data = data_temp

                    else :
                        #if parent is empty and no separator, use last key and
                        key = self._last_key
            
            value = self.cleanup_value_list(key, data)

            self._last_key = key
                
    def get_list_from_headline(self, name):
        self.find_headline(name)
        for headline_start in self._headlines:
            finish = False
            # get html between finded headline and next h[1-6]
            for elt in headline_start.parent.nextSiblingGenerator():
                if elt.name is not None and re.match('h[1-6]', elt.name) and elt.name <= self._headline_tag:
                    break

                if hasattr(elt, 'text'):
                    self.find_list(elt)

    def find_tr(self, table):
        rows = table.findAll('tr')
        for row in rows:
            if len(row.findAll('tr')):
                self.find_tr(row) #recursive
            else:
                index = row.find('th')
                if index is not None:
                    key = index.get_text()
                    value = row.find('td')
                    if value is not None:
                        # replace <br> with \n  to get by .text()
                        value_str = re.sub('</*br/*>', '\n', str(value))
                        value = BeautifulSoup(value_str, "html.parser",  from_encoding='utf-8')
        #                    value = value.get_text().split('\n')
                        value = self.cleanup_value_infobox(key, value.get_text())

        #                     self._result[key] = value


    def get_table(self, class_name='infobox'):
        tables = self._bsObj.findAll('table',{'class':class_name})
        for table in tables:
            self.find_tr(table)
                    
wsc = WikiScraper()
#wsc.load_wiki('朝井リョウと加藤千恵のオールナイトニッポン0(ZERO)', 'ja')
wsc.load_html(p.title, p.html())
# wsc.load_wiki(u'攻殻機動隊', lang='ja', pageid=129204)
# wsc.get_list_from_headline(u'.*スタッフ')
wsc.get_table('infobox')
# print '-----------------------------'
print wsc

勇者ヨシヒコと魔王の城
脚本 : 福田雄一,       
プロデューサー : 浅野太,       
オープニング : mihimaru GT『エボ★レボリューション』,       
監督 : 福田雄一,       
回数 : 12,       
放送時間 : 毎週土曜 0:12 - 0:53,       
出演者 : 木南晴夏,       佐藤二朗,       宅麻伸,       ムロツヨシ,       山田孝之,       岡本あずさ,       
放送枠 : ドラマ24,       
放送分 : 41分,       
放送国・地域 : 日本,       
エンディング : TEE『愛し続けるから』,       
ジャンル : テレビドラマ,       
制作 : テレビ東京,       
放送期間 : 2011年7月9日 - 9月24日,       
音声形式 : ステレオ放送,       



/usr/lib/python2.7/dist-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [3]:
import pandas as pd
data = pd.read_csv("contents.csv",index_col=0)
wscs = []

j = 0
start = 0
stop = start+10#+len(data.index.values)

for i in data.index.values:
    j+=1
    if j<start:
        continue
        
    wsc = WikiScraper()
    wsc.load_wiki(i.decode('utf-8'), pageid=data.at[i, 'id'], lang='ja')
#         wsc.load_wiki(i.decode('utf-8'), 'ja')
# #     wsc.load_html(p.html())
    if wsc._bsObj is not None:
        wsc.get_list_from_headline(u'.*スタッフ')
        wsc.get_table('infobox')
    
    wscs.append(wsc)
    
    print '-------------------------------------'
    print wsc
    print '+++++++++++++++++++++'
    print wsc._name

    if j>=stop:
        break

-------------------------------------
ウンナンさん
音声形式 : ステレオ放送,       
出演者 : 内村光良,       南原清隆,       内村宏幸,       
放送時間 : 火曜 23:55 - 翌0:35,       
放送枠 : Pooh!,       
放送分 : 40分,       
ジャンル : バラエティ番組,       
制作 : TBS,       
放送期間 : 2003年9月30日 - 2004年3月23日,       
放送国・地域 : 日本,       

+++++++++++++++++++++
ウンナンさん
-------------------------------------
ウンナンの気分は上々。
音声形式 : ステレオ放送,       
音響効果 : 江本成治,       
監修 : 星野淳一郎,       
カメラ : 秋本新一,       
VTR編集 : 白澤淳,       反畑弘一,       村上健太郎,       田中直人,       松尾茂樹,       川中健治,       
演出協力 : ウイルスプロダクション,       
オープニング映像 : 小島淳二,       
公開放送 : 松元裕二,       
技術協力 : The TUBE,       東通,       TAMCO,       
制作協力 : 極東電視台,       ホリプロ,       
CAM : 秋本新一,       
ジャンル : バラエティ番組,       
衣裳 : 高野知子,       岩崎孝典,       
演出 : 中川通成,       安田淳,       杉本達,       田口健介,       ほか,       
チーフプロデューサー : 安田淳,       鶴岡滋之,       田代冬彦,       
コーディネーター : Top Taiwan,       
音効 : 江本成治,       
デスク : 椿美貴子,       
編成 : 坂田栄治,       
企画協力 : マセキ芸能社,       
美術プロデューサー : 山口智広,       小畑光良,       河瀬洋

-------------------------------------
ホリさまぁ〜ず
カメラ : 松井光太郎,       
技術協力 : アンサーズ,       八峯テレビ,       
ジャンル : 深夜バラエティ番組,       
音効 : 村松聡,       
TP : 田熊克二,       
構成 : 成瀬正人,       山内正之,       大井洋一,       如月聖也,       
宣伝 : 新名英子,       
メイク : 田沢智美,       
ナレーター : 逸見友惠,       小川はるか,       
プロデューサー : 飯沼美佐子,       
ナレーション : 逸見友惠,       小川はるか,       
演出 : 水野達也,       
VE : 白波孝大,       
美術協力 : TACreate,       
放送時間 : 火曜深夜0:59 - 1:29,       
AP : 久田誠司,       
スタイリスト : 高村純子,       
編集 : 橘本逸人,       
放送期間 : 2009年4月7日 - 2010年3月23日,       
放送国・地域 : 日本,       
MA : 天谷馬直男,       
出演者 : ,       さまぁ〜ず,       青木裕子,       
製作 : TBS,       
ディレクター : 伊達和輝,       塩谷泰孝,       福永勇樹,       高田直,       
音声 : 吉永哲也,       
回数 : 全42,       
放送分 : 30分,       
エンディング : 下記を参照,       
企画 : 大竹一樹,       
制作 : Gyama,       TBS,       

+++++++++++++++++++++
ホリさまぁ〜ず
-------------------------------------
マルさまぁ〜ず
音声形式 : モノラル放送,       
CAM : 松井光太郎,       
技術協力 : アンサーズ,       八峯テレビ,       
ジャンル : 深夜バラエティ番組,       
VE : 高山昌樹,    

In [1]:
#psql -h localhost -U pgadmin -d testdb
#https://tdoc.info/blog/2012/12/05/psycopg2.html
#https://qiita.com/hoto17296/items/0ca1569d6fa54c7c4732
import psycopg2

def get_list_from_list_of_touple(list_data):
    return [x[0] for x in list_data]

def get_decode_list(list_data):
    return [x.decode('utf-8') for x in list_data]

class DBControl(object):
    def __init__(self, dbname, user, password, host='localhost'):
        self._conn = psycopg2.connect(dbname=dbname, host=host, user=user, password=password)
        self._cur = self._conn.cursor()
        
    def __del__(self):
        self._cur.close()
        self._conn.close()

    def execute_commit(self, cur_input):
        self._cur.execute(cur_input)
        self._conn.commit()

    def execute_fetchall(self, cur_input):
        self._cur.execute(cur_input)
        return self._cur.fetchall()
        
    def get_columns_name(self, table):
        cur_input = 'SELECT information_schema.columns.column_name FROM information_schema.columns WHERE table_name=\''+table+'\';'
        return self.execute_fetchall(cur_input)

    def get_table_name(self):
        cur_input = """SELECT table_name FROM information_schema.tables
               WHERE table_schema = 'public'"""
        return self.execute_fetchall(cur_input)

    def create_table(self, table):
        cur_input = 'CREATE TABLE IF NOT EXISTS ' + table + ' (name text PRIMARY KEY);'
        self.execute_commit(cur_input)

    def insert(self, table, column, value):
        if self.get_value(table, column, value) != []:
            print value + ' is exist already'
            return
        
        cur_input = 'INSERT INTO ' + table + ' (' + column + ") VALUES (\'" + value + "\');"
        self.execute_commit(cur_input)
        print value + ' was inserted'

    def update(self, table, column, value, name):
        cur_input = 'UPDATE ' + table + ' set ' + column + '=' + "\'" + value + "\' WHERE name=\'" + name + "\'"
        self.execute_commit(cur_input)

    def alter(self, table, column, data_type):
        cur_input = 'ALTER TABLE '+table+' ADD '+column+' '+data_type+';'
        self.execute_commit(cur_input)
    
    def get_value(self, table, key, value, column='*'):
        cur_input = 'SELECT '+column+' FROM ' + table + ' WHERE ' + key + "=\'" + value + "\';"
        return self.execute_fetchall(cur_input)

import mojimoji, re
def pre_process(data):
    data = data.lower()
    data = data.replace(' ', u'_')
    data = data.replace('-', u'ー')
    data = data.replace(".", u"．")
    data = data.replace(",", u"，")
    data = data.replace("\'", u"’")
    data = data.replace("&", u"＆")
    data = data.replace("/", u"／")
    data = data.replace("(", u"（")
    data = data.replace(")", u"）")
    data = data.replace("[", u"［")
    data = data.replace("]", u"］")
    data = data.replace(":", u"：")
    data = data.replace("!", u"！")
    data = data.replace("?", u"？")
    return data

def key_pre_process(data):
    orig = data
    data = pre_process(data)
    if len(data)>1 and re.match('[0-9]', data[0]):
        d = mojimoji.han_to_zen(data[0].decode('utf-8'))
        data = d+data[1:]

    data = data.encode('utf-8')
#
#     if orig != data:
#         print orig, data
    return data

def value_pre_process(data):
    data = pre_process(data)
    data = data.encode('utf-8')
    return data

In [2]:
#Write python dict to database
#use psycopg not pandas nor sqlalchemy since, column is dynamic
#temporary ignore captial or not
import traceback
                
table_name = 'contents'
dbc = DBControl(dbname='testdb', host='localhost', user='pgadmin', password='password')
dbc.create_table(table_name)
i = 0
# print get_table_name()
for wsc in wscs:
    print i,
    i+=1
    name = value_pre_process(wsc._name) 
    dbc.insert(table_name, 'name', name)
    for k, v in wsc._result.items():
#         print '--------------------------------------'
#         print k, ','.join(set(v))
        k = key_pre_process(k)
        value = value_pre_process(','.join(set(v)))
        if (k,) not in dbc.get_columns_name(table_name):
            try:
                dbc.alter(table_name, k, 'text')
            except:
                print 'error in alter key:', name,  k
                traceback.print_exc()
                dbc._cur.execute('ABORT;')
                continue
        try:
            dbc.update(table_name, k, value, name)
        except:
            print 'error in update value:', name, k, value 
            traceback.print_exc()
            dbc._cur.execute('ABORT;')
            continue
        

NameError: name 'wscs' is not defined

In [ ]:
#https://symfoware.blog.fc2.com/blog-entry-2264.html
#https://qiita.com/ariku/items/75799665acd09520bed2

#create user
#connect as user
#create db

#connect to db
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
url = URL(
    drivername='postgresql',
    username='pgadmin',
    password='password',
    host='localhost',
    database='testdb'
)
engine = create_engine(url, echo=True)

#create table
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy import Column, Integer, String, Float

class Content(Base):
    __tablename__ = 'contents'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    genre = Column(String)
    
    def __repr__(self):
        return "<Contents(name='%s', genre='%s')>" % (
                self.name, self.genre)
    
    @staticmethod
    def add_column(engine, column):
        column_name = column.compile(dialect=engine.dialect)
        column_type = column.type.compile(engine.dialect)
        engine.execute('ALTER TABLE %s ADD COLUMN %s %s' % (Student.__tablename__, column_name, column_type))


Base.metadata.drop_all(bind=engine)
Base.metadata.create_all(engine)

#create session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

#create
new_content = Content(name='test', genre='test_genere')
session.add(new_content)
session.commit()

#read
res = session.query(Content).all()
print res

#close
session.close()

In [46]:
#option one use database
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()
count_once=True
key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

#1. read name and point from csv
input_data = pd.read_csv("contents.csv",index_col=0)
names = input_data.index.values[:5]

#2. get data from database
table_name = 'contents'
dbc = DBControl(dbname='testdb', host='localhost', user='pgadmin', password='password')

#3. separate key and value
#4. calccolumns = dbc.get_columns_name(table_name)
# name = 'ウンナンさん'
for name in names:
    point = float(input_data.at[name, 'ポイント'])
    G.add_node(name, genre='content', point=point)
    attrs = set()

    print '--------',name, point, '----------'
    for column in columns:
        k = column[0]
        res = dbc.get_value(table_name, key='name', value=value_pre_process(name.decode('utf-8')), column=k)
        if len(res)>0:
            val = res[0][0]
            if val is not None:
                v = val.split('，')
                if k == u'ジャンル':
                    if len(v)==0:
                        G.node[wsc._name]['genre'] = v
                    else:
                        G.node[wsc._name]['genre'] = v[0]
                elif k not in key_rm:
                    for attr in v:
                        if attr not in names and attr not in value_rm: #do nothing for contents itself
                            if G.node.get(attr) is None: #first time
                                G.add_node(attr, genre='attribute', point=point)
                            elif not (attr in attrs and count_once): # from second time
                                G.node[attr]['point'] += point
                            G.add_edge(name, attr, relation=k) 
                            attrs.add(attr)

#                 print col, '::', 
#                 for v in val.split('，'):
#                     print v, ' = ',
#                 print
        else:
            pass #todo not in the database

        
#to confirm
# for e in G.edges:
#     print e[0], e[1]
# for n in G.nodes:
#     print n

-------- ウンナンさん 0.2 ----------


/home/kevinh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:36: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/home/kevinh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:41: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
/home/kevinh/.local/lib/python2.7/site-packages/ipykernel_launcher.py:43: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


-------- ウンナンの気分は上々。 0.2 ----------
-------- UN街 0.2 ----------
-------- 勇者ヨシヒコと魔王の城 0.25 ----------
-------- 勇者ヨシヒコと悪霊の鍵 0.25 ----------
曽川修二 ウンナンの気分は上々。
スクウェア・エニックス 勇者ヨシヒコと悪霊の鍵
松元裕二 ウンナンの気分は上々。
堀江利幸 ウンナンの気分は上々。
鹿島雄哉 ウンナンの気分は上々。
手塚公一 勇者ヨシヒコと魔王の城
手塚公一 勇者ヨシヒコと悪霊の鍵
エンディングテーマ：『愛し続けるから』_tee 勇者ヨシヒコと魔王の城
東通 ウンナンの気分は上々。
田代冬彦 ウンナンの気分は上々。
本郷伸明 勇者ヨシヒコと魔王の城
本郷伸明 勇者ヨシヒコと悪霊の鍵
ムロツヨシ 勇者ヨシヒコと魔王の城
ムロツヨシ 勇者ヨシヒコと悪霊の鍵
南原清隆 ウンナンさん
南原清隆 UN街
the_tube ウンナンの気分は上々。
井上大華 ウンナンの気分は上々。
金曜_24：12_ー_24：52 勇者ヨシヒコと悪霊の鍵
椿美貴子 ウンナンの気分は上々。
本田英喜 ウンナンの気分は上々。
内山大輔 ウンナンの気分は上々。
小松幸敏 勇者ヨシヒコと悪霊の鍵
石山貴康 ウンナンの気分は上々。
2003年9月30日_ー_2004年3月23日 ウンナンさん
中野俊成 ウンナンの気分は上々。
中野俊成 UN街
高橋一晃 ウンナンの気分は上々。
高橋一晃 UN街
11 勇者ヨシヒコと悪霊の鍵
勇者ヨシヒコと魔王の城 mihimaru_gt『エボ★レボリューション』
勇者ヨシヒコと魔王の城 本間隆平
勇者ヨシヒコと魔王の城 波多野健
勇者ヨシヒコと魔王の城 テレビ東京
勇者ヨシヒコと魔王の城 2011年7月9日_ー_9月24日
勇者ヨシヒコと魔王の城 及川勝仁
勇者ヨシヒコと魔王の城 武藤大司
勇者ヨシヒコと魔王の城 ステレオ放送
勇者ヨシヒコと魔王の城 「勇者ヨシヒコと魔王の城」製作委員会
勇者ヨシヒコと魔王の城 電通
勇者ヨシヒコと魔王の城 福田雄一
勇者ヨシヒコと魔王の城 佐藤二朗
勇者ヨシヒコと魔王の城 瀬川英史
勇者ヨシヒコと魔王の城 ドラマ24
勇者ヨシヒコと魔王の城 宅麻伸
勇者ヨシヒコと魔王の城 岡部紳二
勇者ヨシヒコと

In [35]:
#option two do not use database

threshold = 1.0
max_number_of_nodes = 500
count_once = True

import networkx as nx
import matplotlib.pyplot as plt

G = nx.Graph()

key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

for wsc in wscs:
    
    point = float(data.at[wsc._name.encode('utf-8'), 'ポイント'])
    G.add_node(wsc._name, genre='content', point=point)
    attrs = set()
    for k, v in wsc._result.items():
        #add first genre to node attribute
        if k == u'ジャンル':
            if len(v)==0:
                G.node[wsc._name]['genre'] = v
            else:
                G.node[wsc._name]['genre'] = v[0]
        elif k not in key_rm:
            for attr in v:
                if attr not in data.index.values and attr not in value_rm: #do nothing for contents itself
                    if G.node.get(attr) is None: #first time
                        G.add_node(attr, genre='attribute', point=point)
                    elif not (attr in attrs and count_once): # from second time
                        G.node[attr]['point'] += point
                    G.add_edge(wsc._name, attr, relation=k) 
                    attrs.add(attr)
    print wsc._name, len(G.nodes)
        
cliques = nx.find_cliques(G)
# #標準出力
# for c in cliques:
#     print c



AttributeError: 'str' object has no attribute 'at'

In [41]:
# remove nodes for visualization
threshold = 1.0
max_number_of_nodes = 500
count_once = True
first = True
while len(G.nodes)>max_number_of_nodes or first:
    print 'Threshold: ', threshold
    print ' node num before simplify:', len(G.nodes)
    value_rm.extend([node for node,degree in dict(G.degree()).items() if degree < 2 and G.node[node]['genre'] is 'attribute']) #remove node based on degree
    value_rm.extend([node for node in G.nodes if  G.node[node]['point'] < threshold and G.node[node]['genre'] is 'attribute'])
    G.remove_nodes_from(value_rm)
    print ' node num after simplify:', len(G.nodes)
    threshold += 0.1
    first = False

#mix the closed genres
for v in names:
    title = v.decode('utf-8')
    if re.search( u'(.*バラエティ.*)|(.*お笑い.*)', G.node[title]['genre']):
        G.node[title]['genre'] = u'バラエティ'
    if re.search( u'.*ドラマ.*', G.node[title]['genre']):
        G.node[title]['genre'] = u'ドラマ'
    if re.search( u'.*SF.*', G.node[title]['genre']):
        G.node[title]['genre'] = u'SF'

nx.draw_networkx(G,font_family='AppleGothic',font_size =8)
# plt.show()
nx.write_gexf(G, 'result.gexf')

Threshold:  1.0
 node num before simplify: 7
 node num after simplify: 7


KeyError: u'\u30a6\u30f3\u30ca\u30f3\u3055\u3093'

In [48]:
#option1
#visualize with pyvis + networkx 
from pyvis.network import Network
# nxg = nx.complete_graph(10)
# gg = Network(notebook=True)
# gg.from_nx(G)
# gg.show("nx.html")

def get_net_node(net, key, value):
    for item in net.nodes:
        if item[key] == value:
            return item
    return None 

net = Network(height='500px', width='1000px')
# net = Network(notebook=True)
net.show_buttons(filter_=['physics'])
net.force_atlas_2based(
    gravity=-30, 
    central_gravity=0.01, 
    spring_length=1.0, 
    spring_strength=0.8, 
    damping=1.0, 
    overlap=1.0)
# net.options.physics.__dict__['timestep'] = 0.5
net.options.physics.__dict__['adaptiveTimestep'] = True
net.toggle_hide_edges_on_drag(True)

scaling = dict(min=1, 
                       max=100, 
                       label=dict(enable=True, 
                                          min=10, 
                                          max=100))

node_id = 0
for label in G.nodes:
#     print label, G.node[label]['genre'], G.node[label]['point']
    if G.node[label]['genre'] == 'attribute':
        shape = 'dot'
        value = G.node[label]['point']
        mass=G.node[label]['point']
    else:
        shape = 'box'
        value = threshold
        mass = 10
    net.add_node(node_id, label=label, 
                 group=G.node[label]['genre'], 
                 value=value,
                 mass=mass,
                 scaling = scaling,
                 shape=shape)
    node_id += 1
    
for edge in G.edges:
    edge[0], edge[1]
    net.add_edge( get_net_node(net, 'label', edge[0])['id'], 
                 get_net_node(net, 'label', edge[1])['id'])
    
# net.save_graph("result.html")
net.show("result.html")

In [ ]:
#opition2
#visualize with pyvis only
from pyvis.network import Network
def get_net_node(net, key, value):
    for item in net.nodes:
        if item[key] == value:
            return item
    return None 

key_rm = [u'放送期間', u'放送時間', u'公開', u'上映時間', u'次作', u'回数', u'放送分']
value_rm = [u'', u'同上', u'日本', u'日本語', u'英語', u'公式サイト', u'ほか', u'ステレオ放送', u'文字多重放送',u'歴代エンディングテーマを参照',
            u'フジテレビ番組基本情報']

net = Network(height='500px', width='1000px')
# net = Network(notebook=True)

count_once = True
node_id = 0
for wsc in wscs:
    point = float(data.at[wsc._name.encode('utf-8'), 'ポイント'])    
    net.add_node(node_id, label=wsc._name, group='content', value=point)
    node_id += 1
    attrs = set()
    #print wsc._name, point
    genre = ''
    for k, v in wsc._result.items():
        #add first genre to node attribute
        if k == u'ジャンル':
            if len(v)==0:
                genre = v
            else:
                genre = v[0]
            get_net_node(net, 'label', wsc._name)['group'] = genre
        elif k not in key_rm:
            for attr in v:
                if attr not in data.index.values and attr not in value_rm: #do nothing for contents itself and value_rm
                    if get_net_node(net, 'label', attr) is None: #first time
                        net.add_node(node_id, label=attr, group='attribute', value=point)
                        node_id += 1

                    elif not (attr in attrs and count_once): # from second time in the same page
                        get_net_node(net, 'label', attr)['value'] += point
                    
                    net.add_edge( get_net_node(net, 'label', wsc._name)['id'], get_net_node(net, 'label', attr)['id'])
                    attrs.add(attr)
    print wsc._name, len(net.nodes)

In [47]:
# remove nodes for visualization
threshold = 1.0
max_number_of_nodes = 500
count_once = True
first = True
while len(net.nodes)>max_number_of_nodes or first:
    print 'Threshold: ', threshold
    print ' node num before simplify:',len(net.nodes)
    net.nodes = [node for node in net.nodes 
                 if (node['value'] > threshold and node['group'] == 'attribute') 
                 or node['group'] != 'attribute']
    print ' node num after simplify:', len(net.nodes)
    threshold += 0.1
    first = False

#mix the closed genres
for v in data.index.values:
    title = v.decode('utf-8')
#     print title, get_net_node(net, 'label', title)
    if re.search( u'(.*バラエティ.*)|(.*お笑い.*)', get_net_node(net, 'label', title)['group']):
        get_net_node(net, 'label', title)['group'] = u'バラエティ'
    if re.search( u'.*ドラマ.*', get_net_node(net, 'label', title)['group']):
        get_net_node(net, 'label', title)['group'] = u'ドラマ'
    if re.search( u'.*SF.*', get_net_node(net, 'label', title)['group']):
        get_net_node(net, 'label', title)['group'] = u'SF'

net.save_graph("test.html")
net.show("test.html")

NameError: name 'net' is not defined